In [0]:
import mlflow
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import numpy as np

In [0]:
mlflow.set_registry_uri("databricks-uc")

In [0]:
EXPERIMENT_NAME = "random-forest-best-models"
RF_PARAMS = ['max_depth', 'n_estimators', 'min_samples_split', 'min_samples_leaf', 'random_state']
mlflow.sklearn.autolog()

In [0]:
# lets use Iris Data
data = load_iris()
X = data.data
y = data.target
print('[+] Data loaded')

X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.6) # Split train data 60%
X_test, X_validation, y_test, y_validation = train_test_split(X_temp, y_temp, train_size=0.5) # Split train and test data to 20% and 20% of original dataset

# Define the parameter space
param_distributions = {
    'n_estimators': [int(x) for x in np.linspace(start=100, stop=1000, num=10)],
    'max_depth': [int(x) for x in np.linspace(10, 100, num=10)] + [None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'random_state': [42]
}
print('[+] Training model')
base_model = RandomForestRegressor()

random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_distributions,
    n_iter=5,
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

with mlflow.start_run(run_name="random_search_optimization"):
    # Fit the random search
    random_search.fit(X_train, y_train)

    # Log the best parameters
    mlflow.log_params(random_search.best_params_)

    # Get predictions and log metrics for best model
    val_predictions = random_search.predict(X_validation)
    test_predictions = random_search.predict(X_test)

    # Log validation and test metrics
    val_rmse = mean_squared_error(y_validation, val_predictions, squared=False)
    test_rmse = mean_squared_error(y_test, test_predictions, squared=False)

    mlflow.log_metric("validation_rmse", val_rmse)
    mlflow.log_metric("test_rmse", test_rmse)

    # Log best score from CV
    mlflow.log_metric("best_cv_score", random_search.best_score_)

    # Log the best model
    mlflow.sklearn.log_model(random_search.best_estimator_, "best_model")

    print('[+] Model trained')

[+] Data loaded
[+] Training model
Fitting 5 folds for each of 5 candidates, totalling 25 fits


2025/05/23 07:24:58 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/databricks/python/lib/python3.11/site-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml"
2025/05/23 07:25:02 INFO mlflow.sklearn.utils: Logging the 5 best runs, no runs will be omitted.
2025/05/23 07:25:08 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


[+] Model trained
